In [1]:
! %module magic necessary so that jupyter fortran kernel knows that 
! the code below is to be compiled as a module and not executable code.
%module: mod_initial
module mod_initial
    
    private
    public :: set_gaussian

contains
    
    pure subroutine set_gaussian(x, icenter, decay)
        real, intent(in out) :: x(:)
        integer, intent(in)  :: icenter
        real, intent(in)     :: decay
        
        integer :: i
    
        do concurrent(i = 1:size(x))
            x(i) = exp(-decay * (i - icenter)**2)
        end do
    end subroutine set_gaussian
    
end module mod_initial

[gfort kernel] module objects created successfully: mod_initial.o

In [2]:
! %module magic necessary so that jupyter fortran kernel knows that 
! the code below is to be compiled as a module and not executable code.
%module: mod_diff 
module mod_diff

    use iso_fortran_env, only: int32, real32
    implicit none
    
    private
    public :: diff
    
contains

    pure function diff(x) result(dx)
        real(real32), intent(in) :: x(:)
        real(real32)             :: dx(size(x))
        integer(int32)           :: im
        
        im = size(x)
        dx(1) = x(1) - x(im)
        dx(2:im) = x(2:im) - x(1:im - 1)
    end function diff
    
end module mod_diff

[gfort kernel] module objects created successfully: mod_diff.o

In [5]:
%fcflags: mod_diff.o mod_initial.o
program tsunami

    use iso_fortran_env, only : int32, real32
    use mod_diff, only        : diff
    use mod_initial, only     : set_gaussian
    
    implicit none
    
    integer(int32) :: n
    
    integer(int32), parameter :: grid_size = 100
    integer(int32), parameter :: num_time_steps = 100
    real(real32),   parameter :: dt = 1, dx = 1, c = 1
    
    real(real32)              :: h(grid_size)
    
    integer(int32), parameter :: icenter = 25
    real(real32), parameter   :: decay = 0.02
    
    if (grid_size <= 0) stop 'grid_size must be > 0'
    if (dt <= 0) stop 'time step dt must be > 0'
    if (dx <= 0) stop 'grid spacing dx must be > 0'
    
    call set_gaussian(h, icenter, decay)
    
    print *, 0, h
    time_loop: do n = 1, num_time_steps
        h = h - c * diff(h) / dx * dt
        print *, n, h
    end do time_loop
        
end program tsunami

           0   9.92950936E-06   2.54193492E-05   6.25215471E-05   1.47748404E-04   3.35462624E-04   7.31802545E-04   1.53381063E-03   3.08871618E-03   5.97602362E-03   1.11089963E-02   1.98410973E-02   3.40474583E-02   5.61347716E-02   8.89216289E-02  0.135335281      0.197898701      0.278037310      0.375311106      0.486752272      0.606530666      0.726149023      0.835270226      0.923116326      0.980198681       1.00000000      0.980198681      0.923116326      0.835270226      0.726149023      0.606530666      0.486752272      0.375311106      0.278037310      0.197898701      0.135335281       8.89216289E-02   5.61347716E-02   3.40474583E-02   1.98410973E-02   1.11089963E-02   5.97602362E-03   3.08871618E-03   1.53381063E-03   7.31802545E-04   3.35462624E-04   1.47748404E-04   6.25215471E-05   2.54193492E-05   9.92950936E-06   3.72665318E-06   1.34381298E-06   4.65571617E-07   1.54975410E-07   4.95640684E-08   1.52299791E-08   4.49635262E-09   1.27540822E-09   3.47589540E-10  